In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keystroke-dynamics-benchmark-data-set/DSL-StrongPasswordData.csv


# **DataSet**

In [2]:
data = pd.read_csv("/kaggle/input/keystroke-dynamics-benchmark-data-set/DSL-StrongPasswordData.csv")
data.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [3]:
for i in range(2,58):
    colname = str(("s00" if i < 10 else "s0")+str(i))
    #print(colname)
    data[colname] = [1 if x ==colname else 0 for x in data['subject']]
data.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,s048,s049,s050,s051,s052,s053,s054,s055,s056,s057
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0,0,0,0,0,0,0,0,0,0
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0,0,0,0,0,0,0,0,0,0
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0,0,0,0,0,0,0,0,0,0
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0,0,0,0,0,0,0,0,0,0
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0,0,0,0,0,0,0,0,0,0


In [4]:
"""
#take data of single subject
sub = 's002'
pos_s2data = data[data['subject']==sub]
pos_s2data = pos_s2data.assign(output=pos_s2data.apply(lambda i: 1 if i['subject']==sub else 0 , axis = 1))

#take data of non-'s002' subjects
neg_s2data = data[data['subject']!=sub]
neg_s2data = neg_s2data.assign(output=neg_s2data.apply(lambda i: 1 if i['subject']==sub else 0 , axis = 1))
neg_s2data = neg_s2data.sample(400)

#concating positive and negative samples to create our final data subset
s2data = pd.concat([pos_s2data, neg_s2data],ignore_index=True)
s2data = s2data.sample(frac=1) #shuffle the data
s2data.head()
"""


"\n#take data of single subject\nsub = 's002'\npos_s2data = data[data['subject']==sub]\npos_s2data = pos_s2data.assign(output=pos_s2data.apply(lambda i: 1 if i['subject']==sub else 0 , axis = 1))\n\n#take data of non-'s002' subjects\nneg_s2data = data[data['subject']!=sub]\nneg_s2data = neg_s2data.assign(output=neg_s2data.apply(lambda i: 1 if i['subject']==sub else 0 , axis = 1))\nneg_s2data = neg_s2data.sample(400)\n\n#concating positive and negative samples to create our final data subset\ns2data = pd.concat([pos_s2data, neg_s2data],ignore_index=True)\ns2data = s2data.sample(frac=1) #shuffle the data\ns2data.head()\n"

In [5]:
data.columns

Index(['subject', 'sessionIndex', 'rep', 'H.period', 'DD.period.t',
       'UD.period.t', 'H.t', 'DD.t.i', 'UD.t.i', 'H.i', 'DD.i.e', 'UD.i.e',
       'H.e', 'DD.e.five', 'UD.e.five', 'H.five', 'DD.five.Shift.r',
       'UD.five.Shift.r', 'H.Shift.r', 'DD.Shift.r.o', 'UD.Shift.r.o', 'H.o',
       'DD.o.a', 'UD.o.a', 'H.a', 'DD.a.n', 'UD.a.n', 'H.n', 'DD.n.l',
       'UD.n.l', 'H.l', 'DD.l.Return', 'UD.l.Return', 'H.Return', 's002',
       's003', 's004', 's005', 's006', 's007', 's008', 's009', 's010', 's011',
       's012', 's013', 's014', 's015', 's016', 's017', 's018', 's019', 's020',
       's021', 's022', 's023', 's024', 's025', 's026', 's027', 's028', 's029',
       's030', 's031', 's032', 's033', 's034', 's035', 's036', 's037', 's038',
       's039', 's040', 's041', 's042', 's043', 's044', 's045', 's046', 's047',
       's048', 's049', 's050', 's051', 's052', 's053', 's054', 's055', 's056',
       's057'],
      dtype='object')

In [6]:
outputcols = ['s002', 's003', 's004', 's005', 's006', 's007', 's008', 's009', 's010',
       's011', 's012', 's013', 's014', 's015', 's016', 's017', 's018', 's019','s020', 's021',
       's022', 's023', 's024', 's025', 's026', 's027', 's028', 's029', 's030', 's031', 's032',
       's033', 's034', 's035', 's036', 's037','s038', 's039', 's040', 's041', 's042', 's043',
       's044', 's045', 's046', 's047', 's048', 's049', 's050', 's051', 's052', 's053', 's054', 's055','s056', 's057']

In [7]:
X = data[['DD.period.t',
       'UD.period.t', 'H.t', 'DD.t.i', 'UD.t.i', 'H.i', 'DD.i.e', 'UD.i.e',
       'H.e', 'DD.e.five', 'UD.e.five', 'H.five', 'DD.five.Shift.r',
       'UD.five.Shift.r', 'H.Shift.r', 'DD.Shift.r.o', 'UD.Shift.r.o', 'H.o',
       'DD.o.a', 'UD.o.a', 'H.a', 'DD.a.n', 'UD.a.n', 'H.n', 'DD.n.l',
       'UD.n.l', 'H.l', 'DD.l.Return', 'UD.l.Return', 'H.Return']]#.to_numpy().reshape(800,10,3)
#print(x_s2[0])
y = data[outputcols]#.to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
#x_s2 = np.asarray(x_s2).astype('float32')

In [8]:
#x_s2train = x_s2[0:600]
#x_s2test = x_s2[600:]
#y_s2train = y_s2[0:600]
#y_s2test = y_s2[600:]
X_train = X_train.to_numpy().reshape(X_train.shape[0],10,3)
X_test = X_test.to_numpy().reshape(X_test.shape[0],10,3)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [12]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, GaussianDropout, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

In [13]:
model = Sequential()
model.add(LSTM(272, input_shape=(10,3), return_sequences='true', kernel_initializer=initializer, activation='tanh'))
model.add(GaussianDropout(0.5))
model.add(LSTM(272, return_sequences='true', activation='tanh'))
model.add(GaussianDropout(0.5))
model.add(LSTM(272, activation='tanh'))
model.add(GaussianDropout(0.5))
model.add(Dense(272, activation='tanh'))
model.add(GaussianDropout(0.5))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss = 'BinaryCrossentropy', optimizer='adam',metrics = ['accuracy'])
#print(model.summary())

batch_size = 32
model.fit(X_train, y_train, epochs = 50, batch_size=batch_size, validation_split = 0.1, verbose = 1)

score,acc = model.evaluate(X_test, y_test, verbose = 1, batch_size = batch_size)
print("score: "+str(score)+" accuracy: "+str(acc))

Epoch 1/50
459/459 [==============================] - 4s 8ms/step - loss: 0.1044 - accuracy: 0.0185 - val_loss: 0.0888 - val_accuracy: 0.0214
Epoch 2/50
459/459 [==============================] - 3s 7ms/step - loss: 0.0904 - accuracy: 0.0297 - val_loss: 0.0803 - val_accuracy: 0.0472
Epoch 3/50
459/459 [==============================] - 3s 6ms/step - loss: 0.0818 - accuracy: 0.0690 - val_loss: 0.0752 - val_accuracy: 0.1189
Epoch 4/50
459/459 [==============================] - 3s 7ms/step - loss: 0.0748 - accuracy: 0.1364 - val_loss: 0.0620 - val_accuracy: 0.3076
Epoch 5/50
459/459 [==============================] - 3s 6ms/step - loss: 0.0618 - accuracy: 0.2944 - val_loss: 0.0494 - val_accuracy: 0.4540
Epoch 6/50
459/459 [==============================] - 3s 7ms/step - loss: 0.0512 - accuracy: 0.4532 - val_loss: 0.0376 - val_accuracy: 0.6324
Epoch 7/50
459/459 [==============================] - 3s 6ms/step - loss: 0.0424 - accuracy: 0.5876 - val_loss: 0.0312 - val_accuracy: 0.6887
Epoch 

In [50]:
sub='s021'
subdata = data[data['subject']==sub] #data.assign(output=data.apply(lambda i: 1 if i['subject']==sub else 0 , axis = 1))
Xsub = subdata[['DD.period.t',
       'UD.period.t', 'H.t', 'DD.t.i', 'UD.t.i', 'H.i', 'DD.i.e', 'UD.i.e',
       'H.e', 'DD.e.five', 'UD.e.five', 'H.five', 'DD.five.Shift.r',
       'UD.five.Shift.r', 'H.Shift.r', 'DD.Shift.r.o', 'UD.Shift.r.o', 'H.o',
       'DD.o.a', 'UD.o.a', 'H.a', 'DD.a.n', 'UD.a.n', 'H.n', 'DD.n.l',
       'UD.n.l', 'H.l', 'DD.l.Return', 'UD.l.Return', 'H.Return']].to_numpy().reshape(subdata.shape[0],10,3)
ysub = subdata[outputcols].to_numpy()
#xtotal = np.asarray(xtotal).astype('float32')

score,acc = model.evaluate(Xsub, ysub, verbose = 1, batch_size = batch_size)
print("score: "+str(score)+" accuracy: "+str(acc))

13/13 [==============================] - 0s 4ms/step - loss: 0.0031 - accuracy: 0.9800
score: 0.003062942298129201 accuracy: 0.9800000190734863


In [53]:
sub='s002'
subdata = data[data['subject']!=sub] #data.assign(output=data.apply(lambda i: 1 if i['subject']==sub else 0 , axis = 1))
Xsub = subdata[['DD.period.t',
       'UD.period.t', 'H.t', 'DD.t.i', 'UD.t.i', 'H.i', 'DD.i.e', 'UD.i.e',
       'H.e', 'DD.e.five', 'UD.e.five', 'H.five', 'DD.five.Shift.r',
       'UD.five.Shift.r', 'H.Shift.r', 'DD.Shift.r.o', 'UD.Shift.r.o', 'H.o',
       'DD.o.a', 'UD.o.a', 'H.a', 'DD.a.n', 'UD.a.n', 'H.n', 'DD.n.l',
       'UD.n.l', 'H.l', 'DD.l.Return', 'UD.l.Return', 'H.Return']].to_numpy().reshape(subdata.shape[0],10,3)
ysub = subdata[outputcols].to_numpy()
#xtotal = np.asarray(xtotal).astype('float32')

predictions = model.predict(Xsub)
pred_df = pd.DataFrame(data = predictions, columns = outputcols)
pred_df["result"] = [1 if x>0.5 else 0 for x in pred_df[sub]]
sum(pred_df["result"])
#pred_df["result"]

9